# Modelo de Recomendación de películas

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

In [2]:
df = pd.read_csv('../clean_data/movies_model.csv')
df.head()

,id_peli,title,word_bag
0,862,Toy Story,Toy Story ToyStoryCollection Animation Comedy ...
1,8844,Jumanji,Jumanji Family Adventure Fantasy JoeJohnston ...
2,15602,Grumpier Old Men,Grumpier Old Men GrumpyOldMenCollection Comedy...
3,31357,Waiting To Exhale,Waiting To Exhale Comedy Romance Drama Forest...
4,11862,Father Of The Bride Part Ii,Father Of The Bride Part Ii Comedy CharlesShy...


In [3]:
cv_fvector = CountVectorizer(stop_words='english')
vector = cv_fvector.fit_transform(df['word_bag']).toarray()
vector.shape

(15076, 125745)

In [4]:
similarity_array = cosine_similarity(vector,vector)
similarity_array.shape

(15076, 15076)

In [6]:
df = pd.read_csv('../clean_data/movies_model.csv')
dfx = pd.DataFrame(columns=['id_peli','idx_recommend'])
for i in df.id_peli:
    recommended_five = sorted(list(enumerate(similarity_array[df[df.id_peli == i].index.item()])),reverse=True,key = lambda x: x[1])[1:6]
    lista = [[i,j[0]] for j in recommended_five]
    dfx = pd.concat([dfx,pd.DataFrame(lista,columns=['id_peli','idx_recommend'])])
    
dfx = dfx.merge(df).drop('word_bag',axis=1)[['id_peli', 'title','idx_recommend']]
dfx.to_csv('../clean_data/recommended_movies.csv')



# Definición de la función para el endpoint final de recomendación.
def recomendacion(pelicula):  # sourcery skip: use-contextlib-suppress
    try:
        pelicula = int(pelicula)
    except ValueError:
        pass
    df = pd.read_csv('../clean_data/movies_model.csv')
    dfx = pd.read_csv('../clean_data/recommended_movies.csv')
    if type(pelicula) == str:
        pelicula = pelicula.title()
        lista = list(dfx[dfx.title == pelicula].idx_recommend)
    elif type(pelicula) == int:
        lista = list(dfx[dfx.id_peli == pelicula].idx_recommend)
        pelicula = dfx.title[dfx.id_peli == pelicula][:1].item()
    else:
        return {"Debes introducir un valor entero (int) que pertenezca al id de alguna película" : "O el nombre de una película en inglés (str)"}

    recomendacion = df.loc[lista][['id_peli', 'title']]
    recomendacion.columns = ['id_peli','recomendaciones']

    if not recomendacion.empty:
        return {
            "Ingresaste la película: ": f"{pelicula}",
            "Las recomendaciones son: ": f"1.- [{recomendacion.recomendaciones[:1].item()}]; 2.- [{recomendacion.recomendaciones[1:2].item()}]; 3.- [{recomendacion.recomendaciones[2:3].item()}]; 4.- [{recomendacion.recomendaciones[3:4].item()}]; 5.- [{recomendacion.recomendaciones[4:5].item()}]",
        }
    total_pelis = len(pd.read_csv('../Data/movies_dataset.csv'))
    return {
        "Dentro del dataset no se encuentra la película: " : f"{pelicula}", 
        "Recuerda que, debido a los procesos de limpieza y preprocesamiento, solo contamos con: " : f"{len(df)} películas", 
        "de un total de: " : f"{total_pelis} películas."}

In [10]:
# Sucedió que en la recomendación de películas, la película misma que es la que debio dar 1, estuviera ordenada en segundo lugar, ya que había otra película diferente con la diferencia = 1,
# lo que no tiene sentido, deben ser películas diferentes menores a 1
recomendacion('star wars')

C:\Users\tinma\AppData\Local\Temp\ipykernel_6808\1422992557.py:36: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  "Las recomendaciones son: ": f"1.- [{recomendacion.recomendaciones[:1].item()}]; 2.- [{recomendacion.recomendaciones[1:2].item()}]; 3.- [{recomendacion.recomendaciones[2:3].item()}]; 4.- [{recomendacion.recomendaciones[3:4].item()}]; 5.- [{recomendacion.recomendaciones[4:5].item()}]",


{'Ingresaste la película: ': 'Star Wars',
 'Las recomendaciones son: ': '1.- [The Empire Strikes Back]; 2.- [The Star Wars Holiday Special]; 3.- [Empire Of Dreams: The Story Of The Star Wars Trilogy]; 4.- [Return Of The Jedi]; 5.- [Robot Chicken: Star Wars]'}